In [ ]:
# 3.0 LLMs and Chat Models
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

llm.predict("How many planets are there?")
chat.predict("How many planets are there?")

In [ ]:
# 3.1 Predict Messages with Chat Model

from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

chat = ChatOpenAI(temperature=0.1)

messages = [
    SystemMessage(content="You are Madalena Bay. Reply as if you were her."),
    AIMessage(content="It's me, Imaginal Disk"),
    HumanMessage(content="What songs you are influenced by?")
]

chat.predict_messages(messages)

In [ ]:
# 3.2 Customize prompt with Prompt Template

from langchain.chat_models import  ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What songs did Magdalena Bay listen when making {track}?"
)
prompt = template.format(track="Imaginal Disk")

chat.predict(prompt)

In [ ]:
# 3.2 Customize prompt with Chat Prompt Template

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    (
        "human",
        "What is the distance between {country_a} and {country_b}. Also, what is your name?",
    ),
])
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

In [ ]:
# 3.3 Concept of OutputParser

from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("It's ,you, the,  purest,you")

In [ ]:
# 3.3 Apply OutputParser

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

chat = ChatOpenAI(temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ("system",
    """
     You are a list generating machine. 
     Answer question with these conditions.
     - COMMA SEPERATED LIST
     - List have maximum {max_items} items.
     - All characters in lowercase
    """),
    ("human","{question}")
])
prompt = template.format_messages(
    max_items=5,
    question="What are the pokemons?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

In [ ]:
# 3.3 OutputParser and LCEL

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(temperature=0.1)
template = ChatPromptTemplate.from_messages([
    (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
    ),
    ("human", "{question}")
])
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

chain = template | chat | CommaOutputParser()
chain.invoke({"max_items":5, "question":"What are the pokemons?"})

In [ ]:
# 3.4 Chaining Chains

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()], # listening to events on LLM
)
original_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a pro baker. You create a easy to follow recipe that anyone can succeed with."),
    ("human", "I want to bake {bread_type}.")
])
original_chain = original_prompt | chat

modify_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a Korean baker. You change the original baking recipe to Koreans love. You add and adjust some ingredients. If you don't know how to convert, just say so."),
    ("human", "{recipe}")
])
modify_chain = modify_prompt | chat

final_chain = {"recipe": original_chain} | modify_chain
# modify_chain.invoke({"recipe": original_chain_result})

final_chain.invoke({"bread_type": "Country Bread"})